# Semantic Search using ELSER v2 text expansion

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/03-ELSER.ipynb)


Learn how to use the [ELSER](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html) for text expansion-powered semantic search.

**`Note:`** This notebook demonstrates how to use ELSER model `.elser_model_2` model which offers an improved retrieval accuracy. 

If you have set up an index with ELSER model `.elser_model_1`, and would like to upgrade to ELSER v2 model - `.elser_model_2`, Please follow instructions from the notebook on [how to upgrade an index to use elser model](../model-upgrades/upgrading-index-to-use-elser.ipynb)

# Install and Connect

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the following packages:

- `elasticsearch`


In [ ]:
!pip install -qU elasticsearch

Next, we need to import the modules we need.
🔐 NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

In [96]:
from elasticsearch import Elasticsearch, helpers, exceptions
from urllib.request import urlopen
import getpass
import json
import time

Now we can instantiate the Python Elasticsearch client.

First we prompt the user for their password and Cloud ID.
Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [179]:
# Found in the 'Manage Deployment' page
CLOUD_ID = getpass.getpass('Elastic Cloud ID: ')

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass('Elastic password: ')

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

Confirm that the client has connected with this test

In [173]:
print(client.info())

{'name': 'instance-0000000000', 'cluster_name': '591083df1d524a0eb0299c617d5a093f', 'cluster_uuid': 'pwoEM8ElQoCMGgfKLAkgVQ', 'version': {'number': '8.10.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'e338da74c79465dfdc204971e600342b0aa87b6b', 'build_date': '2023-09-07T08:16:21.960703010Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


# Download and Deploy ELSER Model

In this example, we are going to download and deploy the ELSER model in our ML node. Make sure you have an ML node in order to run the ELSER model.

In [180]:
# delete model if already downloaded and deployed
try:
  client.ml.delete_trained_model(model_id=".elser_model_2",force=True)
  print("Model deleted successfully, We will proceed with creating one")
except exceptions.NotFoundError:
  print("Model doesn't exist, but We will proceed with creating one")

# Creates the ELSER model configuration. Automatically downloads the model if it doesn't exist. 
client.ml.put_trained_model(
    model_id=".elser_model_2",
    input={
      "field_names": ["text_field"]
    }
  )


Model doesn't exist, but We will proceed with creating one


ObjectApiResponse({'model_id': '.elser_model_2', 'model_type': 'pytorch', 'model_package': {'packaged_model_id': 'elser_model_2', 'model_repository': 'https://ml-models.elastic.co', 'minimum_version': '11.0.0', 'size': 438123914, 'sha256': '2e0450a1c598221a919917cbb05d8672aed6c613c028008fedcd696462c81af0', 'metadata': {}, 'tags': [], 'vocabulary_file': 'elser_model_2.vocab.json'}, 'created_by': 'api_user', 'version': '11.0.0', 'create_time': 1698438246925, 'model_size_bytes': 0, 'estimated_operations': 0, 'license_level': 'platinum', 'description': 'Elastic Learned Sparse EncodeR v2', 'tags': ['elastic'], 'metadata': {}, 'input': {'field_names': ['text_field']}, 'inference_config': {'text_expansion': {'vocabulary': {'index': '.ml-inference-native-000002'}, 'tokenization': {'bert': {'do_lower_case': True, 'with_special_tokens': True, 'max_sequence_length': 512, 'truncate': 'first', 'span': -1}}}}, 'location': {'index': {'name': '.ml-inference-native-000002'}}})

The above command will download the ELSER model. This will take a few minutes to complete. Use the following command to check the status of the model download.

In [181]:
while True:
    status = client.ml.get_trained_models(
        model_id=".elser_model_2",
        include="definition_status"
    )
    
    if (status["trained_model_configs"][0]["fully_defined"]):
        print("ELSER Model is downloaded and ready to be deployed.")
        break
    else:
        print("ELSER Model is downloaded but not ready to be deployed.")
    time.sleep(5)

ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded and ready to be deployed.


Once the model is downloaded, we can deploy the model in our ML node. Use the following command to deploy the model.

In [ ]:
# Start trained model deployment if not already deployed
client.ml.start_trained_model_deployment(
  model_id=".elser_model_2",
  number_of_allocations=1
)


This also will take a few minutes to complete.

# Indexing Documents with ELSER

In order to use ELSER on our Elastic Cloud deployment we'll need to create an ingest pipeline that contains an inference processor that runs the ELSER model.
Let's add that pipeline using the [`put_pipeline`](https://www.elastic.co/guide/en/elasticsearch/reference/master/put-pipeline-api.html) method.

In [ ]:
client.ingest.put_pipeline(
    id="elser-ingest-pipeline", 
    description="Ingest pipeline for ELSER",
    processors=[
    {
      "inference": {
        "model_id": ".elser_model_2",
        "input_output": [
            {
              "input_field": "plot",
              "output_field": "plot_embedding"
            }
          ]
      }
    }
  ]
)

Let's note a few important parameters from that API call:

- `inference`: A processor that performs inference using a machine learning model.
- `model_id`: Specifies the ID of the machine learning model to be used. In this example, the model ID is set to `.elser_model_2`.
- `input_output`: Specifies input and output fields
- `input_field`: Field name from which the `sparse_vector` representation are created.
- `output_field`:  Field name which contains inference results. 

## Create index

To use the ELSER model at index time, we'll need to create an index mapping that supports a [`text_expansion`](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-text-expansion-query.html) query.
The mapping includes a field of type [`sparse_vector`](https://www.elastic.co/guide/en/elasticsearch/reference/master/sparse-vector.html)  to work with our feature vectors of interest.
This field contains the token-weight pairs the ELSER model created based on the input text.

Let's create an index named `elser-example-movies` with the mappings we need.


In [162]:
client.indices.delete(index="elser-example-movies", ignore_unavailable=True)
client.indices.create(
  index="elser-example-movies",
  settings={
      "index": {
          "number_of_shards": 1,
          "number_of_replicas": 1,
          "default_pipeline": "elser-ingest-pipeline"
      }
  },
  mappings={
    "properties": {
      "plot": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "plot_embedding": { 
        "type": "sparse_vector" 
      }
    }
  }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'elser-example-movies'})

## Insert Documents
Let's insert our example dataset of 12 movies.

If you get an error, check the model has been deployed and is available in the ML node. In newer versions of Elastic Cloud, ML node is autoscaled and the ML node may not be ready yet. Wait for a few minutes and try again.

In [163]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json"
response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

# Prepare the documents to be indexed
documents = []
for doc in data_json:
    documents.append({
        "_index": "elser-example-movies",
        "_source": doc,
    })

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `elser-example-movies` index!")
time.sleep(3)

Done indexing documents into `elser-example-movies` index!


Inspect a new document to confirm that it now has an `plot_embedding` field that contains a list of new, additional terms.
These terms are the **text expansion** of the field(s) you targeted for ELSER inference in `input_field` while creating the pipeline. 
ELSER essentially creates a tree of expanded terms to improve the semantic searchability of your documents.
We'll be able to search these documents using a `text_expansion` query.

But first let's start with a simple keyword search, to see how ELSER delivers semantically relevant results out of the box.

# Searching Documents

Let's test out semantic search using ELSER.

In [164]:
response = client.search(
    index='elser-example-movies', 
    size=3,
    query={
        "text_expansion": {
            "plot_embedding": {
                "model_id":".elser_model_2",
                "model_text":"fighting movie"
            }
        }
    }
)

for hit in response['hits']['hits']:
    doc_id = hit['_id']
    score = hit['_score']
    title = hit['_source']['title']
    plot = hit['_source']['plot']
    print(f"Score: {score}\nTitle: {title}\nPlot: {plot}\n")

Score: 12.763346
Title: Fight Club
Plot: An insomniac office worker and a devil-may-care soapmaker form an underground fight club that evolves into something much, much more.

Score: 9.930427
Title: Pulp Fiction
Plot: The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.

Score: 9.4883375
Title: The Matrix
Plot: A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.



## Next Steps
Now that we have a working example of semantic search using ELSER, you can try it out on your own data. Don't forget to scale down the ML node when you are done. 